## Modeling

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from collections import defaultdict
import time
import pickle

# Check device
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

print(f"Using device: {device}")

with open('cifar10_loaders.pkl', 'rb') as f:
    data = pickle.load(f)

train_loader = data['train_loader']
val_loader = data['val_loader']
test_loader = data['test_loader']
class_names = data['class_names']
batch_size = data['batch_size']

print("DataLoaders loaded successfully!")
print(f"Training batches: {len(train_loader)}")


Using device: mps
DataLoaders loaded successfully!
Training batches: 625
